# Migrate the Database From MySQL to MariaDB Cloud (SkySQL)
# Setup HTAP Replication, Insert data to Row Store and Auto Replicate to Column Store

Here we are actually using cloud MariaDB, the MariaDB Platform optimized for smart transaction processing in the cloud,
storing data both as rows on persistent block storage and as columns on object storage.
It can practically use as data warehouse depending on the setup.

We can specify the Row Store tables with certain prefix to automatically replicate to Column Store. Which can feed data
directly to the datawarehouse.

Because the mariadb connector is not working properly locally, we used mysql.connector to connect to the cloud database
which Mariadb supports. And the cloud server is call SkySQL.

In [1]:
import pandas as pd
from IPython.display import display
import mysql.connector

In [2]:
def create_connection():
    conn = None
    try:
       conn = mysql.connector.connect(
          user="DB00004175",
          password="tTNlXEirHh)EmwXhtp4koTR",
          host="menu-data-db.mdb0002067.db.skysql.net",
          ssl_ca="skysql_chain.pem",
          port=5001)
       conn.autocommit = False
       return conn
    except mysql.connector.Error as e:
       print(f"Error connecting to MariaDB Platform: {e}")
    return conn

conn = create_connection()

In [3]:
# For DDL, DCL, DML
def sql_execute(conn, sql):

    try:
        cur = conn.cursor()
        cur.execute(sql, multi=True)
        conn.commit()
    except mysql.connector.Error as e:
        print(e)

In [4]:
# For DQL
def sql_execute_show(conn, sql):

    try:
        df = pd.read_sql(sql, con = conn)
        display(df)
    except mysql.connector.Error as e:
        print(e)

In [9]:
### create tables
def create_table_main(conn):
    sql_create_restaurant_table = """CREATE TABLE IF NOT EXISTS menu_rowstore.restaurant (
                                        r_id INT PRIMARY KEY,
                                        r_name VARCHAR(200) NOT NULL
                                    ); """

    sql_create_food_category_table = """CREATE TABLE IF NOT EXISTS menu_rowstore.food_category (
                                    cat_id INT PRIMARY KEY,
                                    cat_name VARCHAR(200) NOT NULL
                                );"""

    sql_create_item_info_table = """CREATE TABLE IF NOT EXISTS menu_rowstore.item_info (
                                    item_id INT NOT NULL,
                                    year INT NOT NULL,
                                    item_name VARCHAR(200) NOT NULL,
                                    item_description text,
                                    PRIMARY KEY (item_id, year)
                                );"""

    sql_create_nutrition_facts_table = """CREATE TABLE IF NOT EXISTS menu_rowstore.nutrition_facts (
                                    item_id INT NOT NULL,
                                    year INT NOT NULL,
                                    r_id INT,
                                    cat_id INT,
                                    calories INT,
                                    total_fat INT,
                                    saturated_fat INT,
                                    trans_fat INT,
                                    cholesterol INT,
                                    sodium INT,
                                    potassium INT,
                                    carbs INT,
                                    protein INT,
                                    sugar INT,
                                    dietary_Fiber INT,
                                    PRIMARY KEY (item_id, year)
                                    #FOREIGN KEY (item_id) REFERENCES item_info(item_id),
                                    #FOREIGN KEY (year) REFERENCES item_info(year),
                                    #FOREIGN KEY (r_id) REFERENCES restaurant(r_id),
                                    #FOREIGN KEY (cat_id) REFERENCES food_category(cat_id)
                                );"""

    sql_create_combo_facts_table = """CREATE TABLE IF NOT EXISTS menu_rowstore.combo_facts (
                                    combo_id INT NOT NULL,
                                    item_id INT NOT NULL,
                                    year INT NOT NULL,
                                    cat_id INT,
                                    builds text,
                                    PRIMARY KEY (combo_id, item_id, year, cat_id)
                                    #FOREIGN KEY (`item_id`) REFERENCES `item_info`(`item_id`),
                                    #FOREIGN KEY (`year`) REFERENCES `item_info`(`year`),
                                    #FOREIGN KEY (`cat_id`) REFERENCES `food_category`(`cat_id`)
                                );"""


    # create a database connection
    #conn = create_connection()

    # create tables
    if conn is not None:
        # create tables
        try:
            sql_execute(conn, sql_create_restaurant_table)
            sql_execute(conn, sql_create_food_category_table)
            sql_execute(conn, sql_create_item_info_table)
            sql_execute(conn, sql_create_nutrition_facts_table)
            sql_execute(conn, sql_create_combo_facts_table)
        except:
            print("Error! cannot create the database connection.")
        finally:
            if conn:
                conn.close()



# Used MysqlWorkbench to connect to Mariadb SkySQL to import the mysql dump file over to the cloud

Use the credentials provided in the MariaDB Enterprise Server to connect to the server

### Create a database in the server

In [ ]:
sql_execute(conn, sql = '''CREATE DATABASE IF NOT EXISTS menu_rstore;;''')

In [15]:
sql_execute_show(conn, sql = '''SELECT * FROM menu_rstore.restaurant;''')

,r_id,r_name
0,0,7 Eleven
1,1,Applebee's
2,2,Arby's
3,3,Auntie Anne's
4,4,BJ's Restaurant & Brewhouse
...,...,...
91,91,Whataburger
92,92,White Castle
93,93,Wingstop
94,94,Yard House


# Setup HTAP Replication, Row Store Auto Replicate to Column Store
(Hybrid Transactional-Analytical Processing)

Reference: https://mariadb.com/products/skysql/docs/operations/htap-replication/

## Create a analytical (column store) database


#### 1. Set up the replication filter using the set_htap_replication() UDF:

In [16]:
sql_execute(conn,
            sql = '''
            SELECT set_htap_replication(
              'menu_rowstore.[[:word:]]+',
              'menu_rowstore',
              'menu_cstore');
            ;''')

#### 2. Create the transactional and analytical databases:

In [17]:
sql_execute(conn,
            sql = '''
            CREATE DATABASE IF NOT EXISTS menu_rowstore;
            CREATE DATABASE IF NOT EXISTS menu_cstore;
            ''')

#### 3. Create the transactional tables that will be replicated:

In [ ]:
create_table_main(conn)

#### 4. Confirm that the corresponding analytical tables were created:

In [20]:
sql_execute_show(conn, '''SHOW TABLES FROM menu_cstore;''')

,Tables_in_menu_cstore
0,combo_facts
1,food_category
2,item_info
3,nutrition_facts
4,restaurant


#### 5. The replication initially creates empty menu_rowstore tables,
#### which need to be transformed into menu_cstore tables and which need to be populated
#### with the initial copy of the data:
- First need to drop all the tables created earlier
- Recreate tables using the columnstore engine

In [10]:
sql_execute(conn,
            sql = '''
            CREATE TABLE IF NOT EXISTS menu_cstore.restaurant
            ENGINE = COLUMNSTORE
            SELECT * FROM menu_rowstore.restaurant
            ;
            CREATE TABLE IF NOT EXISTS menu_cstore.food_category
            ENGINE = COLUMNSTORE
            SELECT * FROM menu_rowstore.food_category
            ;
            CREATE TABLE IF NOT EXISTS menu_cstore.item_info
            ENGINE = COLUMNSTORE
            SELECT * FROM menu_rowstore.item_info
            ;
            CREATE TABLE IF NOT EXISTS menu_cstore.nutrition_facts
            ENGINE = COLUMNSTORE
            SELECT * FROM menu_rowstore.nutrition_facts
            ;
            CREATE TABLE IF NOT EXISTS menu_cstore.combo_facts
            ENGINE = COLUMNSTORE
            SELECT * FROM menu_rowstore.combo_facts
            ;
            ''')

MySQL Connection not available.


#### 6. Insert data into the transactional tables that will be replicated:
- Imported .sql dump file which was exported from MySQL server to the menu_rowstore database
- Using MySQLWorkbench


#### 7. Confirm that the data was replicated to the corresponding analytical tables:

In [5]:
sql_execute_show(conn, sql = '''SELECT * FROM menu_cstore.restaurant;''')
sql_execute_show(conn, sql = '''SELECT * FROM menu_cstore.food_category;''')
sql_execute_show(conn, sql = '''SELECT COUNT(year) FROM menu_cstore.item_info;''')
sql_execute_show(conn, sql = '''SELECT COUNT(year) FROM menu_cstore.nutrition_facts;''')
sql_execute_show(conn, sql = '''SELECT COUNT(year) FROM menu_cstore.combo_facts;''')


,r_id,r_name
0,0,7 Eleven
1,1,Applebee's
2,2,Arby's
3,3,Auntie Anne's
4,4,BJ's Restaurant & Brewhouse
...,...,...
91,91,Whataburger
92,92,White Castle
93,93,Wingstop
94,94,Yard House


,cat_id,cat_name
0,0,Appetizers & Sides
1,1,Baked Goods
2,2,Beverages
3,3,Burgers
4,4,Desserts
5,5,Entrees
6,6,Fried Potatoes
7,7,Pizza
8,8,Salads
9,9,Sandwiches


,COUNT(year)
0,209532


,COUNT(year)
0,209532


,COUNT(year)
0,131332


## Testing querying with the Column Store

In [6]:
sql_execute_show(conn, sql = '''
SELECT year, AVG(calories), AVG(total_fat), AVG(Protein) FROM menu_cstore.nutrition_facts
GROUP BY year
;
''')

,year,AVG(calories),AVG(total_fat),AVG(Protein)
0,2008,477.7172,26.1575,18.9965
1,2010,369.4070,17.5817,12.5405
2,2012,439.7074,32.7203,24.1088
3,2013,370.0542,16.1378,13.4993
4,2014,375.3779,16.7372,13.4871
5,2015,407.4768,18.4459,15.2967
6,2016,378.0453,16.8043,14.0246
7,2017,374.0719,16.9589,13.9133
8,2018,386.7493,17.4970,14.7468


In [28]:
conn.close()

# DataBase setup completed!

